In [2]:

import pandas as pd

In [3]:
def signal_generator(df):
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    Pre_open = df.Open.iloc[-2]
    Pre_close = df.Close.iloc[-2]

    #Bearish Signal
    if (open > close and Pre_close>Pre_open and
        close < Pre_open and Pre_close<=open):
        return 1
    
    #Bullish signal
    elif (
        close>open and Pre_close<Pre_open and
        Pre_open<close and Pre_close>= open):
        return 2
    
    #no signal
    else:
        return 0


In [14]:
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import MarketOrderRequest, TakeProfitRequest, StopLossRequest
from alpaca.trading.enums import OrderSide, TimeInForce, OrderClass
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime

API_KEY = 'your api key'
SECRET_KEY = 'your secret key'
client = TradingClient(API_KEY, SECRET_KEY)

In [34]:

def get_candle(n):
    data_client = StockHistoricalDataClient(API_KEY, SECRET_KEY)
    request_para = StockBarsRequest(
        symbol_or_symbols='AAPL',
        feed= 'iex',
        timeframe=TimeFrame.Minute ,
        start=datetime(2025, 1,1),
        end=datetime.now()
    )

    bars = data_client.get_stock_bars(request_para)
    df = bars.df.reset_index()
    df = df.drop('symbol', axis=1)
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df = df.set_index('timestamp')
    candles = df.resample('15T').agg({
        'open': 'first',
        'high' : 'max',
        'low':'min',
        'close':'last',
        'volume':'sum',
    }).dropna()
    return candles.sort_values(by='timestamp', ascending=False).head(n)
candle = get_candle(3)
candle

C:\Users\AMIT\AppData\Local\Temp\ipykernel_16288\3964695889.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


,open,high,low,close,volume
timestamp,,,,,
2025-07-08 13:30:00+00:00,210.01,210.455,209.70,209.86,21212.0
2025-07-07 19:45:00+00:00,209.45,210.170,209.34,209.94,86714.0
2025-07-07 19:30:00+00:00,209.47,209.550,209.22,209.43,44900.0


In [ ]:
def trading_job():
    candle = get_candle(3)

    new_df = pd.DataFrame({
        'Open' : candle['open'][:3],
        'Close' : candle['close'][:3],
        'High' : candle['high'][:3],
        'Low' : candle['low'][:3],
    })

    new_df['Open'] = new_df['Open'].astype(float)
    new_df['Close'] = new_df['Close'].astype(float)
    new_df['High'] = new_df['High'].astype(float)
    new_df['Low'] = new_df['Low'].astype(float)

    signal = signal_generator(new_df.iloc[1:,:])
    
    pnl_Ratio = 2
    pre_candleR = abs(new_df['High'].iloc[1]- new_df['Low'].iloc[1])

    SLbuy = new_df['Open'].iloc[0] - pre_candleR
    SLbuy = round(SLbuy,2)
    SLsell = new_df['Open'].iloc[0] + pre_candleR

    TPbuy = new_df['Open'].iloc[0] + pre_candleR*pnl_Ratio
    TPsell = new_df['Open'].iloc[0] - pre_candleR*pnl_Ratio
    TPbuy = round(TPbuy,2)
    TPsell = round(TPsell,2)
    SLsell = round(SLsell,2)

    print(new_df.iloc[1:,:])
    print(f"TPbuy : {TPbuy}", " ", f"SLbuy: {SLbuy}", " ", f"TPsell : {TPsell}", " ", f"SLsell : {SLsell}")
    #sell
    if signal == 1:
        mo= MarketOrderRequest(
                    symbol="AAPL",
                    qty=100,
                    side=OrderSide.SELL,
                    time_in_force=TimeInForce.DAY,
                    order_class=OrderClass.BRACKET,
                    take_profit=TakeProfitRequest(limit_price = TPsell),
                    stop_loss=StopLossRequest(stop_price=SLsell)
                    )

        bracket_order = client.submit_order(order_data=mo)
    
    #Buy
    elif signal == 2:
        mo= MarketOrderRequest(
                    symbol="AAPL",
                    qty=100,
                    side=OrderSide.BUY,
                    time_in_force=TimeInForce.DAY,
                    order_class=OrderClass.BRACKET,
                    take_profit=TakeProfitRequest(limit_price=TPbuy),
                    stop_loss=StopLossRequest(stop_price=SLbuy)
                    )

        bracket_order = client.submit_order(order_data=mo)

import time

while True:
    try:
        trading_job()
    except Exception as e:
        print("Error:", e)
    
    time.sleep(60)  # Run every 60 seconds




C:\Users\AMIT\AppData\Local\Temp\ipykernel_16288\3964695889.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


                             Open   Close     High     Low
timestamp                                                 
2025-07-08 13:30:00+00:00  210.01  209.02  210.455  208.51
2025-07-07 19:45:00+00:00  209.45  209.94  210.170  209.34
TPbuy : 212.91   SLbuy: 207.08   TPsell : 205.13   SLsell : 210.97


C:\Users\AMIT\AppData\Local\Temp\ipykernel_16288\3964695889.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


                             Open   Close     High     Low
timestamp                                                 
2025-07-08 13:30:00+00:00  210.01  209.02  210.455  208.51
2025-07-07 19:45:00+00:00  209.45  209.94  210.170  209.34
TPbuy : 212.91   SLbuy: 207.08   TPsell : 205.13   SLsell : 210.97


C:\Users\AMIT\AppData\Local\Temp\ipykernel_16288\3964695889.py:16: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  candles = df.resample('15T').agg({


                             Open   Close     High     Low
timestamp                                                 
2025-07-08 13:30:00+00:00  210.01  209.02  210.455  208.51
2025-07-07 19:45:00+00:00  209.45  209.94  210.170  209.34
TPbuy : 212.91   SLbuy: 207.08   TPsell : 205.13   SLsell : 210.97
